In [ ]:
#Subir Votes_SinDuplicar y PostA_SinDuplicar a HDFS
# importar hdfs
from hdfs import InsecureClient

#conectarme
client = InsecureClient('http://localhost:9870/', user='usuarioFAI')

#Borrar
borrar = client.delete('/so/PostA_SinDuplicar.csv',recursive=True)
borrar = client.delete('/so/PostQ_SinDuplicar.csv',recursive=True)

#Subir un archivo
subirPostsA = client.upload('/so/','/home/usuarioFAI/PostA_SinDuplicar.csv')
subirPostsQ = client.upload('/so/','/home/usuarioFAI/PostQ_SinDuplicar.csv')
#subirVotes = client.upload('/so/','/home/usuarioFAI/Votes_SinDuplicar.csv')

# Listar todos los archivos de una carpeta.
listado = client.list('/so/')
print(listado)

In [1]:
#Segunda etapa de filtrado trabajamos solo con PostQ_SinDuplicar
#importar la libreria
from pyspark.sql import SparkSession

In [2]:
#abrir una sesion con un nombre
spark=SparkSession.builder.appName('pysarkpruebas').getOrCreate()

In [3]:
#leer archivo de PostQ_SinDuplicar (Questions) de HDFS
#header true es para que la primera fila sea considerada los atributos y no una tupla mas
#inferschema es para que tome los tipos de datos como son, no todo string..prueben poner false y luego imprimir el esquema
dfq = spark.read.csv('hdfs://127.0.0.1:9000/so/PostQ_sin_body_ni_title.csv', header=True,  inferSchema=False)
dfq.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- PostTypeId: string (nullable = true)
 |-- AcceptedAnswerId: string (nullable = true)
 |-- CreationDate: string (nullable = true)
 |-- Tags: string (nullable = true)
 |-- AnswerCount: string (nullable = true)



In [4]:

#total de preguntas
total = dfq.count()
dfq_completo = dfq
print('Cantidad preguntas Iniciales', total)

#Sacamos las preguntas con al menos una respuestas
dfq1 = dfq.filter(dfq['AnswerCount'] > 0) 
cant_preg1 = dfq1.count()
print('Cantidad preguntas con al menos una respuesta', cant_preg1)

#Sacamos los Preguntas que tengan tags nulos
dfq2 = dfq.filter(dfq.Tags.isNull()) 
cant_preg2= dfq2.count()
print('Cantidad preguntas con Tags no nulos', cant_preg2)

#Sacamos las preguntas no aceptadas
dfq3 = dfq.filter(dfq.AcceptedAnswerId.isNull())
cant_preg3= dfq3.count()
print('Cantidad Preguntas sin Respuesta Aceptada', cant_preg3)

Cantidad preguntas Iniciales 145623
Cantidad preguntas con al menos una respuesta 111622
Cantidad preguntas con Tags no nulos 0
Cantidad Preguntas sin Respuesta Aceptada 84037


In [ ]:
dfq_completo.show(5)

In [5]:
#Dejamos una tabla final con todos los filtros aplicados
df = dfq_completo.filter(dfq_completo['AnswerCount'] > 0) 
df.show(5)
df = df.filter(df.Tags.isNotNull())
df.show(5)
df = df.filter(df.AcceptedAnswerId.isNotNull())
df.show(5)
cant_preg = df.count()
print('Cantidad preguntas Finales', cant_preg)

+---+---+----------+----------------+--------------------+--------------------+-----------+
|_c0| Id|PostTypeId|AcceptedAnswerId|        CreationDate|                Tags|AnswerCount|
+---+---+----------+----------------+--------------------+--------------------+-----------+
|  0|  1|         1|               2|2015-10-29T15:56:...|<.net><asp.net-we...|          1|
|  1|  3|         1|               9|2015-10-29T23:54:...|            <delphi>|          3|
|  2|  5|         1|             208|2015-10-30T01:15:...|       <c++><string>|          7|
|  3|  6|         1|             654|2015-10-30T01:36:...|      <c++><casting>|          2|
|  4|  8|         1|             442|2015-10-30T10:26:...|<google-maps><ang...|          1|
+---+---+----------+----------------+--------------------+--------------------+-----------+
only showing top 5 rows

+---+---+----------+----------------+--------------------+--------------------+-----------+
|_c0| Id|PostTypeId|AcceptedAnswerId|        CreationDa

In [6]:
import pandas as pd

Titulos = ['Cantidad preguntas con al menos una respuesta', 'Cantidad preguntas con Tags no nulos', 
           'Cantidad Preguntas sin Respuesta Aceptada', 'Cantidad preguntas Iniciales','Cantidad preguntas Finales']
Cantidades = []

Cantidades.append(cant_preg1) #asi con cada cant
Cantidades.append(cant_preg2) #asi con cada cant
Cantidades.append(cant_preg3)
Cantidades.append(total) #asi con cada cant
Cantidades.append(cant_preg) #asi con cada cant

dfres = pd.DataFrame({'Descripción':Titulos, 'Cantidades':Cantidades})

#df.show() #verificar si funciona

print(dfres)
#lo guardamos 
dfres.to_csv('resumen_parte2.csv') 

                                     Descripción  Cantidades
0  Cantidad preguntas con al menos una respuesta      111622
1           Cantidad preguntas con Tags no nulos           0
2      Cantidad Preguntas sin Respuesta Aceptada       84037
3                   Cantidad preguntas Iniciales      145623
4                     Cantidad preguntas Finales       61586


In [7]:
#guardamos PostQ_Final en un csv
df.toPandas().to_csv('PostQ_Final.csv')

In [8]:
print(df.count())

61586
